In [7]:

pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
pd.read_csv('firebase_data.csv')

,test7@gmail_com,chhhd,2024-03-11 05:53:14.752370Z
0,test7@gmail_com,the,2024-03-11 05:54:31.371945Z
1,test7@gmail_com,value,2024-03-11 05:54:35.228342Z
2,test7@gmail_com,id,2024-03-11 05:54:35.696709Z
3,test7@gmail_com,grester,2024-03-11 05:54:37.621267Z
4,test7@gmail_com,thst,2024-03-11 05:54:38.195503Z
5,test7@gmail_com,wht,2024-03-11 05:54:39.259525Z
6,test7@gmail_com,your,2024-03-11 05:54:40.014854Z
7,test7@gmail_com,thinging,2024-03-11 05:54:42.394930Z
8,test7@gmail_com,baby,2024-03-11 05:56:39.392282Z
9,test7@gmail_com,fhrhri,2024-03-11 05:57:15.707415Z


In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define the dataset
class MoodDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.word_to_idx = {word: idx for idx, word in enumerate(set(data['words']), start=1)}
        self.word_to_idx['<unk>'] = 0  # Add the '<unk>' token with index 0
        self.mood_to_idx = {'happiness': 0, 'sadness': 1, 'depression': 2}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = self.data.iloc[idx]['words']
        mood = self.data.iloc[idx]['mood']
        return self.word_to_idx.get(word, 0), self.mood_to_idx[mood]

# Define the neural network model
class MoodClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(MoodClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        out = self.fc1(embedded.view(-1, embedded.size(-1)))
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_model(data, epochs=100, batch_size=32, lr=0.001, embedding_dim=100, hidden_dim=128):
    # Load the dataset
    dataset = MoodDataset(data)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Instantiate the model
    vocab_size = len(dataset.word_to_idx)
    output_dim = len(dataset.mood_to_idx)
    model = MoodClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)

    # Train the model
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        for words, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(words)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for words, labels in test_loader:
            outputs = model(words)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100 * correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

    return model

def predict_moods(model, dataset, word_lists):
    mood_to_label = {0: 'happiness', 1: 'sadness', 2: 'depression'}
    
    for words in word_lists:
        word_indices = [dataset.word_to_idx.get(word, 0) for word in words]
        inputs = torch.LongTensor(word_indices)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        predicted_moods = [mood_to_label[prediction.item()] for prediction in predicted]
        print(f"Words: {', '.join(words)}")
        
        mood_counts = {
            'happiness': predicted_moods.count('happiness'),
            'sadness': predicted_moods.count('sadness'),
            'depression': predicted_moods.count('depression')
        }
        
        average_mood = max(mood_counts, key=mood_counts.get)
        print(f"\nAverage Predicted Mood: {average_mood}")
        print()

# Example usage
data = pd.read_csv('hinglish_emotion_dataset.csv')
model = train_model(data)
dataset = MoodDataset(data)

words1 = ['khush', 'maza', 'hansi', 'pyaar', 'anand', 'khushiyaan']
words2 = ['dukhi', 'udaas', 'ronaa', 'gham', 'malaal', 'vishad']
words3 = ['hangover', 'bore', 'tension', 'stress', 'boring', 'sucide']
words4 = ['excitement', 'masti', 'funnie', 'hasnaa', 'maza aaya', 'majedaar']
words5 = ['paagal', 'lajawab', 'kasht', 'dukh', 'emotional', 'aansu']

word_lists = [words1, words2, words3, words4, words5]

predict_moods(model, dataset, word_lists)

Test Loss: 0.1861, Test Accuracy: 18.75%
Words: khush, maza, hansi, pyaar, anand, khushiyaan

Average Predicted Mood: happiness

Words: dukhi, udaas, ronaa, gham, malaal, vishad

Average Predicted Mood: sadness

Words: hangover, bore, tension, stress, boring, sucide

Average Predicted Mood: sadness

Words: excitement, masti, funnie, hasnaa, maza aaya, majedaar

Average Predicted Mood: happiness

Words: paagal, lajawab, kasht, dukh, emotional, aansu

Average Predicted Mood: sadness



In [4]:
cd ..


/config/workspace


In [1]:
# retriving sepecific person data
df =

In [63]:

import firebase_admin
from firebase_admin import credentials, auth, firestore ,db
from datetime import datetime, timedelta
import pandas as pd


# Initialize Firebase Admin SDK
cred = credentials.Certificate("perfectkey-bc35f-firebase-adminsdk-mj5eo-2d903258ae.json")
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://perfectkey-bc35f-default-rtdb.firebaseio.com/'
# })
# Get a reference to the Firebase Realtime Database
ref = db.reference()
try:
   # Get email from query parameters
         # Get a reference to the 'users' node
       users_ref = ref.child('users')
       print(users_ref)

    # Initialize an empty list to store user data
       user_data = []

        # Get all user emails
       user_emails = users_ref.get().keys()
       
       print(user_emails)

       email_tst = 'test7@gmail_com'
       for email in user_emails:
        words_ref = users_ref.child(email).child('words')
        print(words_ref)

          # Get all words for this user
        words_data = words_ref.get()
        print(words_data)

        
          # Append user data to the list
        user_data.append({
                'email': email,
                'words': words_data.values() if words_data else []
          })
        u_D = pd.DataFrame(user_data)
       u_D.head()
       print(user_data)

except Exception as e:
       print(e)

dict_keys(['test7@gmail_com', 'test@gmail_com'])
{'-Nsg3cEzlXUENlm02AWu': {'timestamp': '2024-03-11 05:53:14.752370Z', 'word': 'chhhd'}, '-Nsg3ux7nCTIldWoTgBi': {'timestamp': '2024-03-11 05:54:31.371945Z', 'word': 'the'}, '-Nsg3vtRQp4G2EKU-smy': {'timestamp': '2024-03-11 05:54:35.228342Z', 'word': 'value'}, '-Nsg3w-khar4i4YEBjzY': {'timestamp': '2024-03-11 05:54:35.696709Z', 'word': 'id'}, '-Nsg3wTpPBQL9Yow538Z': {'timestamp': '2024-03-11 05:54:37.621267Z', 'word': 'grester'}, '-Nsg3wbnZB0p6w6Ie-Lt': {'timestamp': '2024-03-11 05:54:38.195503Z', 'word': 'thst'}, '-Nsg3wsQV6Llk_tVw2cO': {'timestamp': '2024-03-11 05:54:39.259525Z', 'word': 'wht'}, '-Nsg3x3DLp8fWyptCIfr': {'timestamp': '2024-03-11 05:54:40.014854Z', 'word': 'your'}, '-Nsg3xdP8LSjN_tDyqkJ': {'timestamp': '2024-03-11 05:54:42.394930Z', 'word': 'thinging'}, '-Nsg4PCUAgBcvUFGXXCS': {'timestamp': '2024-03-11 05:56:39.392282Z', 'word': 'baby'}, '-Nsg4Y3vZmLEKkKYU4yK': {'timestamp': '2024-03-11 05:57:15.707415Z', 'word': 'fhrhri'

Testing the app code 

In [8]:
import sys
sys.path.append('src')

import pandas as pd
import torch
from torch.utils.data import DataLoader
from datasets import MoodDataset
from models import MoodClassifier, train_model, save_model
from utils import setup_logging, log_exception

def main():
    setup_logging()

    # Load data
    data = pd.read_csv('hinglish_emotion_dataset.csv')
    dataset = MoodDataset(data)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Train model
    model = MoodClassifier(len(dataset.word_to_idx), 100, 128, 3)
    train_model(model, train_loader, test_loader)

    # Save model
    save_model(model, 'models/mood_classifier.pth')

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        log_exception(e)


Epoch 1/10, Test Loss: 0.0616, Test Accuracy: 68.75%
Epoch 2/10, Test Loss: 0.0597, Test Accuracy: 68.75%
Epoch 3/10, Test Loss: 0.0579, Test Accuracy: 75.00%
Epoch 4/10, Test Loss: 0.0564, Test Accuracy: 68.75%
Epoch 5/10, Test Loss: 0.0551, Test Accuracy: 68.75%
Epoch 6/10, Test Loss: 0.0538, Test Accuracy: 68.75%
Epoch 7/10, Test Loss: 0.0528, Test Accuracy: 81.25%
Epoch 8/10, Test Loss: 0.0518, Test Accuracy: 81.25%
Epoch 9/10, Test Loss: 0.0509, Test Accuracy: 81.25%
Epoch 10/10, Test Loss: 0.0500, Test Accuracy: 75.00%


In [113]:
r = typed_words('test7@gmail_com')
pd.DataFrame(r['words'].iloc[0])

,timestamp,word
0,2024-03-11 05:53:14.752370Z,chhhd
1,2024-03-11 05:54:31.371945Z,the
2,2024-03-11 05:54:35.228342Z,value
3,2024-03-11 05:54:35.696709Z,id
4,2024-03-11 05:54:37.621267Z,grester
5,2024-03-11 05:54:38.195503Z,thst
6,2024-03-11 05:54:39.259525Z,wht
7,2024-03-11 05:54:40.014854Z,your
8,2024-03-11 05:54:42.394930Z,thinging
9,2024-03-11 05:56:39.392282Z,baby


In [11]:
from flask import Flask, render_template, request, redirect, url_for
import firebase_admin
from firebase_admin import credentials, auth, db
from datetime import datetime, timedelta
import pandas as pd

cred = credentials.Certificate("perfectkey-bc35f-firebase-adminsdk-mj5eo-2d903258ae.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://perfectkey-bc35f-default-rtdb.firebaseio.com/'
})
ref = db.reference()

def typed_words1(email):
    try:
        users_ref = ref.child('users')

        # Initialize an empty list to store user data
        all_user_data = []

        # Get all user emails
        all_user_emails = users_ref.get().keys()

        for user_email in all_user_emails:
            words_ref = users_ref.child(user_email).child('words')
            
            # Get all words for this user
            words_data = words_ref.get()
            
            # Append user data to the list
            all_user_data.append({
                'email': user_email,
                'words': words_data.values() if words_data else []
            })
            print(all_user_data)

        a_u_d = pd.DataFrame(all_user_data)
        print('all_user_data : ________________________________________\n' )
        
        # email = request.args.get('email')
        print('email___________________\n' , email)  # Get email from query parameters
        row = a_u_d[a_u_d['email'] == email]
        print('row __________________________________________________________________\n' , row)

        return render_template('typed_words.html', row=row)

    except Exception as e:
        print(e)


In [17]:
h = typed_words1(email)
h = pd.DataFrame(h)

[{'email': 'test7@gmail_com', 'words': dict_values([{'timestamp': '2024-03-11 05:53:14.752370Z', 'word': 'chhhd'}, {'timestamp': '2024-03-11 05:54:31.371945Z', 'word': 'the'}, {'timestamp': '2024-03-11 05:54:35.228342Z', 'word': 'value'}, {'timestamp': '2024-03-11 05:54:35.696709Z', 'word': 'id'}, {'timestamp': '2024-03-11 05:54:37.621267Z', 'word': 'grester'}, {'timestamp': '2024-03-11 05:54:38.195503Z', 'word': 'thst'}, {'timestamp': '2024-03-11 05:54:39.259525Z', 'word': 'wht'}, {'timestamp': '2024-03-11 05:54:40.014854Z', 'word': 'your'}, {'timestamp': '2024-03-11 05:54:42.394930Z', 'word': 'thinging'}, {'timestamp': '2024-03-11 05:56:39.392282Z', 'word': 'baby'}, {'timestamp': '2024-03-11 05:57:15.707415Z', 'word': 'fhrhri'}, {'timestamp': '2024-03-11 05:57:16.691166Z', 'word': 'hjis'}, {'timestamp': '2024-03-11 05:57:17.755889Z', 'word': 'ieiiei'}, {'timestamp': '2024-03-11 05:57:18.957826Z', 'word': 'nsiwooej'}, {'timestamp': '2024-03-11 05:57:20.026878Z', 'word': 'nrhri'}, {'ti

In [18]:
h.head()

""


In [2]:
email = 'test@gmail_com'
row = df[df['email'] == email]
col =row['words'].iloc[0]
col = pd.DataFrame(col)
col.head()

NameError: name 'df' is not defined

In [10]:
cd ..

/config/workspace


## modular function to retrive data of specific email from firebase

In [2]:
def typed_words(email):
  try:
            users_ref = ref.child('users')

            # Initialize an empty list to store user data
            all_user_data = []

              # Get all user emails
            all_user_emails = users_ref.get().keys()
            
            print(all_user_emails)

            for user_email in all_user_emails:
                  words_ref = users_ref.child(user_email).child('words')
                  print(words_ref)

                    # Get all words for this user
                  words_data = words_ref.get()
                  # print(words_data)

                  
                    # Append user data to the list
                  all_user_data.append({
                          'email': user_email,
                          'words': words_data.values() if words_data else []
                    })

            all_user_data = pd.DataFrame(all_user_data)
            print(all_user_data)
            email = email.replace('.', '_')
            row = all_user_data[all_user_data['email'] == email]
            return row
      
      #  print(user_data)

  except Exception as e:
       print(e)
    

In [3]:
# Mood Analyzer Class
class MoodAnalyzer:
    def __init__(self, data):
        self.data = data
        print('data')

    def analyze_mood(self):
        model = train_model(self.data)
        print('model')
        word_lists = [self.data['word'].iloc[i].split() for i in range(len(self.data))]
        print('word_lists')
        print(word_lists)
        predicted_moods = predict_moods(model, self.data, word_lists)
        print('1')
        return predicted_moods

In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define the dataset
class MoodDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.word_to_idx = {word: idx for idx, word in enumerate(set(data['words']), start=1)}
        self.word_to_idx['<unk>'] = 0  # Add the '<unk>' token with index 0
        self.mood_to_idx = {'happiness': 0, 'sadness': 1, 'depression': 2}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = self.data.iloc[idx]['words']
        mood = self.data.iloc[idx]['mood']
        print('7')
        return self.word_to_idx.get(word, 0), self.mood_to_idx[mood]

# Define the neural network model
class MoodClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(MoodClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        out = self.fc1(embedded.view(-1, embedded.size(-1)))
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_model(data, epochs=100, batch_size=32, lr=0.001, embedding_dim=100, hidden_dim=128):
    # Load the dataset
    dataset = MoodDataset(data)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    print('8')

    # Instantiate the model
    vocab_size = len(dataset.word_to_idx)
    output_dim = len(dataset.mood_to_idx)
    model = MoodClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)

    # Train the model
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        for words, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(words)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for words, labels in test_loader:
            outputs = model(words)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100 * correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

    return model

def predict_moods(model, dataset, word_lists):
    print('predicit_moods')
    mood_to_label = {0: 'happiness', 1: 'sadness', 2: 'depression'}
    
    for words in word_lists:
        word_indices = [dataset.word_to_idx.get(word, 0) for word in words]
        inputs = torch.LongTensor(word_indices)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        predicted_moods = [mood_to_label[prediction.item()] for prediction in predicted]
        print(f"Words: {', '.join(words)}")
        
        mood_counts = {
            'happiness': predicted_moods.count('happiness'),
            'sadness': predicted_moods.count('sadness'),
            'depression': predicted_moods.count('depression')
        }
        
        average_mood = max(mood_counts, key=mood_counts.get)
        print(f"\nAverage Predicted Mood: {average_mood}")
        print()

        

# # Example usage
# data = pd.read_csv('hinglish_emotion_dataset.csv')
# model = train_model(data)

# words1 = ['khush', 'maza', 'hansi', 'pyaar', 'anand', 'khushiyaan']
# words2 = ['dukhi', 'udaas', 'ronaa', 'gham', 'malaal', 'vishad']
# words3 = ['hangover', 'bore', 'tension', 'stress', 'boring', 'sucide']
# words4 = ['excitement', 'masti', 'funnie', 'hasnaa', 'maza aaya', 'majedaar']
# words5 = ['paagal', 'lajawab', 'kasht', 'dukh', 'emotional', 'aansu']

# word_lists = [words1, words2, words3, words4, words5]

# dataset = MoodDataset(data)
# predict_moods(model, dataset, word_lists)